## Load data and install packages

In [69]:
import pandas as pd
import re
from tqdm import tqdm
tqdm.pandas()

In [ ]:
Code_df = pd.read_csv('D:/Th/data/KGT_with_no_output/KGT_codes.csv')

In [13]:
Code_df.head()

,Unnamed: 0,project_name,project_ID,cell_index,source,output_type,execution_count
0,0,kerneler_starter-portrait-segmentation-b08b391...,11413234,3.0,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,NaN,1.0
1,1,kerneler_starter-portrait-segmentation-b08b391...,11413234,5.0,"for dirname, _, filenames in os.walk('/kaggle/...",stream,2.0
2,2,kerneler_starter-portrait-segmentation-b08b391...,11413234,7.0,# Distribution graphs (histogram/bar graph) of...,NaN,3.0
3,3,kerneler_starter-portrait-segmentation-b08b391...,11413234,8.0,# Correlation matrix\ndef plotCorrelationMatri...,NaN,4.0
4,4,kerneler_starter-portrait-segmentation-b08b391...,11413234,9.0,# Scatter and density plots\ndef plotScatterMa...,NaN,5.0


In [26]:
len(Code_df)

5072599

In [2]:
Code_df.columns

Index(['Unnamed: 0', 'project_name', 'project_ID', 'cell_index', 'source',
       'output_type', 'execution_count'],
      dtype='object')

## clculate code cell metrics

In [38]:
Code_df['LOC'] = Code_df['source'].apply(lambda x: x.count('\n')+1 if type(x)==str else 0)

In [39]:
Code_df['BLC'] = Code_df['LOC'].apply(lambda x:1 if x==0 else 0)

In [55]:
Code_df['UDF'] = Code_df['source'].apply(lambda x: sum([len(re.findall('^(?!#).*def ', y)) for y in x.split('\n')]) if type(x)==str else 0)

In [59]:
Code_df['I'] = Code_df['source'].apply(lambda x: x.count('import ') if type(x)==str else 0)

In [58]:
Code_df['EH'] = Code_df['source'].apply(lambda x: x.count('try:')  if type(x)==str else 0)

In [61]:
import statistics as sts
Code_df['ALLC'] = Code_df['source'].apply(lambda x: sts.mean([len(y) for y in x.split('\n')]) if type(x)==str else 0)

In [70]:
Code_df['NDD'] = Code_df['output_type'].progress_apply(lambda x: x.count('display_data') if type(x)==str else 0)

100%|████████████████████████████████████████████████████████████████████| 5072599/5072599 [00:09<00:00, 517620.82it/s]


In [71]:
Code_df['NEC'] = Code_df['output_type'].progress_apply(lambda x: x.count('execute_result') if type(x)==str else 0)

100%|████████████████████████████████████████████████████████████████████| 5072599/5072599 [00:09<00:00, 543870.83it/s]


In [75]:
Code_df.columns

Index(['Unnamed: 0', 'project_name', 'project_ID', 'cell_index', 'source',
       'output_type', 'execution_count', 'LOC', 'BLC', 'UDF', 'I', 'EH',
       'ALLC', 'NDD', 'NEC'],
      dtype='object')

Extract number of operator

In [193]:
#extract number of python operators from text
def extract_operator_count(text):
    operators = [
        '+', '-', '*', '/', '**', '%', '//',
        '=', '+=', '-=', '*=', '/=', '%=', '//=', '**=',
        '&=', '|=', '^=', '>>=', '<<=',
        '==', '!=', '>=', '<=', '>', '<',
        ' and ', ' or ', ' not ',
        ' is ', ' is not ',
        ' in ', ' not in ',
        '&', '|', '^', '~', '<<', '>>',

    ]
    count = 0
    for operator in operators:
        count += text.count(operator)
    return count

def extract_unique_operator_count(text):
    operators = [
        '+', '-', '*', '/', '**', '%', '//',
        '=', '+=', '-=', '*=', '/=', '%=', '//=', '**=',
        '&=', '|=', '^=', '>>=', '<<=',
        '==', '!=', '>=', '<=', '>', '<',
        ' and ', ' or ', ' not ',
        ' is ', ' is not ',
        ' in ', ' not in ',
        '&', '|', '^', '~', '<<', '>>',

    ]
    unique_count = 0
    for operator in operators:
        if text.count(operator) > 0:
            unique_count += 1
    return unique_count
    

#https://www.w3schools.com/python/python_operators.asp

In [194]:
#remove file address from string
def remove_file_address(text): 
    text = re.sub(r'\S*[\\|/]\S*', '', text)
    return text

def remove_ampersand(text):
    text = re.sub(r'%[a-zA-Z]+', '', text)
    return text

def remove_extras(text):
    text = remove_ampersand(text)
    text = remove_file_address(text)
    return text

Keywords count

In [195]:
#extract python keywords from text
def extract_keywords_count(text):
    keywords = ['True', 'False', 'None', 'and', 'as', 'assert', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except', 'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'nonlocal', 'not', 'or', 'pass', 'raise', 'return', 'try', 'while', 'with', 'yield']
    count = 0
    for keywords in keywords:
        count += text.count(keywords)
    return count

Identifiers

In [196]:
keywords = ['True', 'False', 'None', 'and', 'as', 'assert', 'break', 'class', 'continue', 'def', 'del', 'elif', 'else', 'except', 'finally', 'for', 'from', 'global', 'if', 'import', 'in', 'is', 'lambda', 'nonlocal', 'not', 'or', 'pass', 'raise', 'return', 'try', 'while', 'with', 'yield']

def extract_identifier_count(text):
    count = 0
    identifiers = re.findall(r'[a-zA-Z_][a-zA-Z0-9_]*', text)
    for identifier in identifiers:
        if identifier not in keywords:
            count += 1
    return count

In [197]:
def extract_avg_len_identifier(text):
    iden_lens = []
    identifiers = re.findall(r'[a-zA-Z_][a-zA-Z0-9_]*', text)
    for identifier in identifiers:
        if identifier not in keywords:
            iden_lens.append(len(identifier))
    if len(iden_lens) == 0:
        return 0
    return sum(iden_lens) // len (iden_lens)

def extract_max_len_identifier(text):
    iden_lens = []
    identifiers = re.findall(r'[a-zA-Z_][a-zA-Z0-9_]*', text)
    for identifier in identifiers:
        if identifier not in keywords:
            iden_lens.append(len(identifier))
    if len(iden_lens) == 0:
        return 0
    return max(iden_lens)

KLCID

In [198]:
def extract_unique_lines_of_code(text):
    lines = text.split('\n')
    return set(lines)

def klcid(text):
    unique_lines_code = extract_unique_lines_of_code(text)
    iden_count = []
    for line in unique_lines_code:
        iden_count.append(extract_identifier_count(line))
    unique_lines_code_with_iden = []
    for line in unique_lines_code:
        if extract_identifier_count(line) > 0:
            unique_lines_code_with_iden.append(line)
    if len(unique_lines_code_with_iden) == 0:
        return 0
    return sum(iden_count) / len(unique_lines_code_with_iden)


# Klemola, T. & Rilling, Juergen. (2003). A cognitive complexity metric based on category learning.

Extract operands

In [199]:
def extract_operand_count(string):
    res = re.split(r'''[\-()\+\*\/\=\&\|\^\~\<\>\%]|
    \*\*|\/\/|
    \>\>|\<\<|
    \+\=|\-\=|\*\=|\/\=|\%\=|
    \/\/\=|\*\*\=
    \&\=|\|\=|\^\=|\>\>\=|\>\>\=|
    \=\=|\!\=|\>\=''', string)
    res = list(filter(None, res))
    return len(res)



In [200]:
def extract_unique_operand_count(string):
    res = re.split(r'''[\-()\+\*\/\=\&\|\^\~\<\>\%]|
    \*\*|\/\/|
    \>\>|\<\<|
    \+\=|\-\=|\*\=|\/\=|\%\=|
    \/\/\=|\*\*\=
    \&\=|\|\=|\^\=|\>\>\=|\>\>\=|
    \=\=|\!\=|\>\=''', string)
    res = list(filter(None, res))
    return len(set(res))

Count python argument

In [201]:
s = 'def salam(a,b,c):\n    return a+b+c def salam2(x,y,z):\n    return a+b+c'
# def python_arguments(string):
#     args = []
#     args_regex = re.compile(
#             r'''(
#                 def*\W(\w+)([\w\W]*?)\((.*?)\)
#             )''',
#             flags=re.DOTALL | re.VERBOSE | re.MULTILINE
#         )

#     s = re.findall(args_regex, string)
#     z = [i[0] for i in s]
#     for i in z:
#         args = args + re.search(r'\((.*?)\)',i).group(1).split(',')
#     return args



def python_arguments(string):
    args = []
    args_regex = re.compile(
            r'''(
                [a-zA-Z_][a-zA-Z0-9_]*\((.*?)\)
            )''',
            flags=re.DOTALL | re.VERBOSE | re.MULTILINE
        )
    try:
        s = re.findall(args_regex, string)
        z = [i[0] for i in s]
        for i in z:
            args = args + re.search(r'\((.*?)\)',i).group(1).split(',')
        return len(args)
    except:
        return 0

Loop and condition statements

In [205]:
def extract_loop_statements_count(text):
    keywords = ['while', 'for']
    count = 0
    for keywords in keywords:
        count += text.count(keywords)
    return count

def extract_if_statements_count(text):
    keywords = ['if']
    count = 0
    for keywords in keywords:
        count += text.count(keywords)
    return count

def statements_count(text):
    return extract_loop_statements_count(text) + extract_if_statements_count(text)

nested depth block

In [210]:
def nested_depth(text):
    blocks = text.split('\n')
    depth = []
    for block in blocks:
        depth.append(block.count(' '))
    return sum(depth) / len(depth)

In [ ]:
import radon

from radon.visitors import ComplexityVisitor
def complexity_analysis(source):
    v = ComplexityVisitor.from_code(source)
    return v.complexity
def complexity_analysis2(source):
    source_new = source.replace('%matplotlib inline', '')
    try:
        v = ComplexityVisitor.from_code(source_new)
        return v.complexity
    except Exception as e:
        # print(e)
        return 0

External API Popularity

In [ ]:
def capture_imports(source_code):
    import_regex = r"^\s*(?:from|import)\s+(\w+(?:\s*,\s*\w+)*)"
    # Find all import statements
    import_matches = re.findall(import_regex, source_code, re.MULTILINE)
    for i in import_matches:
        if ',' in i:
            new_i = i.replace(' ', '').split(',')
            import_matches.extend(new_i)
            import_matches.remove(i)
    return import_matches
code_df['API'] = code_df['source'].apply(lambda x: capture_imports(str(x))) 

In [ ]:
import collections

l = code_df['API'].values.tolist()
flat_list = [item for sublist in l for item in sublist]
eap_dict = dict(collections.Counter(flat_list))
eap_score_dict = dict(sorted(eap_dict.items(), key=lambda item: item[1], reverse=True))
max_freq = eap_dict['sklearn']
for k, v in eap_score_dict.items():
    eap_score_dict[k] = v / max_freq
def eap_score(ap_list):
    score = 0
    for i in ap_list:
        score += eap_score_dict.get(i, 0)
    return score
code_df['EAP'] = code_df['API'].apply(lambda x: eap_score(set(x)))   

In [206]:
Code_df['S'] = Code_df['source'].progress_apply(lambda x: statements_count(str(x)))

100%|████████████████████████████████████████████████████████████████████| 5072599/5072599 [00:24<00:00, 208959.55it/s]


In [207]:
Code_df['P'] = Code_df['source'].progress_apply(lambda x: python_arguments(str(x)))


100%|█████████████████████████████████████████████████████████████████████| 5072599/5072599 [03:47<00:00, 22342.44it/s]


In [208]:
Code_df['KLCID'] = Code_df['source'].progress_apply(lambda x: klcid(str(x)))


100%|██████████████████████████████████████████████████████████████████████| 5072599/5072599 [08:32<00:00, 9890.18it/s]


In [211]:
Code_df['NBD'] = Code_df['source'].progress_apply(lambda x: nested_depth(str(x)))


100%|████████████████████████████████████████████████████████████████████| 5072599/5072599 [00:37<00:00, 134052.85it/s]


In [213]:
Code_df['OPRND'] = Code_df['source'].progress_apply(lambda x: extract_operand_count(str(x)))


100%|█████████████████████████████████████████████████████████████████████| 5072599/5072599 [03:15<00:00, 26003.97it/s]


In [214]:
Code_df.columnsumns

Index(['Unnamed: 0', 'project_name', 'project_ID', 'cell_index', 'source',
       'output_type', 'execution_count', 'LOC', 'BLC', 'UDF', 'I', 'EH',
       'ALLC', 'NDD', 'NEC', 'S', 'P', 'KLCID', 'NBD', 'OPRND'],
      dtype='object')

In [215]:
Code_df['OPRATOR'] = Code_df['source'].progress_apply(lambda x: extract_operator_count(str(x)))


100%|█████████████████████████████████████████████████████████████████████| 5072599/5072599 [01:47<00:00, 47378.36it/s]


In [216]:
Code_df['UOPRND'] = Code_df['source'].progress_apply(lambda x: extract_unique_operand_count(str(x)))


100%|█████████████████████████████████████████████████████████████████████| 5072599/5072599 [03:25<00:00, 24658.24it/s]


In [217]:
Code_df['UOPRATOR'] = Code_df['source'].progress_apply(lambda x: extract_unique_operator_count(str(x)))


100%|█████████████████████████████████████████████████████████████████████| 5072599/5072599 [01:44<00:00, 48648.13it/s]


In [218]:
Code_df['ID'] = Code_df['source'].progress_apply(lambda x: extract_identifier_count(str(x)))


100%|█████████████████████████████████████████████████████████████████████| 5072599/5072599 [03:11<00:00, 26476.25it/s]


In [219]:
Code_df['ALID'] = Code_df['source'].progress_apply(lambda x: extract_avg_len_identifier(str(x)))


100%|█████████████████████████████████████████████████████████████████████| 5072599/5072599 [03:31<00:00, 23945.08it/s]


In [220]:
Code_df['MLID'] = Code_df['source'].progress_apply(lambda x: extract_max_len_identifier(str(x)))


100%|█████████████████████████████████████████████████████████████████████| 5072599/5072599 [03:34<00:00, 23649.18it/s]


In [ ]:
code_df ['CyC'] = code_df['source'].progress_apply(lambda x: complexity_analysis2(str(x)))


## comment metrics

#### Count inline comments

In [221]:
def count_inline_comment(string: str) -> str:
    inline_regex = re.compile(
        r'''(
            (?<=\#).+ # comments like: # This is a comment
        )''',
        flags=re.VERBOSE
    )
    # return inline_regex.sub('', string)

    return len(re.findall(inline_regex, string))

def multi_line_comments(string: str) -> str:

    # Python comments
    multi_line_python_regex = re.compile(
        r'''(
            (?<=\n)\'{3}.*?\'{3}(?=\s*\n) |
            (?<=^)\'{3}.*?\'{3}(?=\s*\n) |
            (?<=\n)\'{3}.*?\'{3}(?=$) |
            (?<=^)\'{3}.*?\'{3}(?=$) |
            (?<=\n)\"{3}.*?\"{3}(?=\s*\n) |
            (?<=^)\"{3}.*?\"{3}(?=\s*\n) |
            (?<=\n)\"{3}.*?\"{3}(?=$) |
            (?<=^)\"{3}.*?\"{3}(?=$)
        )''',
        flags=re.DOTALL | re.VERBOSE | re.MULTILINE
    )
    python_multi_line_count = re.findall(multi_line_python_regex, string)


    return python_multi_line_count

def extract_line_comments(text):
    multi_lines_lines = []
    multi_lines = multi_line_comments(text)
    for line in multi_lines:
        multi_lines_lines += line.split('\n')
    return len(multi_lines_lines) + count_inline_comment(text) 
    
def count_comment_word(text):
    comment_word_count = 0
    comments = re.findall(r'(?<=\#).+', text)
    for comment in comments:
        comment_word_count += len(comment.split())
    return comment_word_count

In [222]:
Code_df['LOCom'] = Code_df['source'].progress_apply(lambda x: extract_line_comments(str(x)))


100%|██████████████████████████████████████████████████████████████████████| 5072599/5072599 [09:22<00:00, 9014.44it/s]


In [223]:
Code_df['CW'] = Code_df['source'].progress_apply(lambda x: count_comment_word(str(x)))


100%|█████████████████████████████████████████████████████████████████████| 5072599/5072599 [00:57<00:00, 87685.67it/s]


In [224]:
Code_df.head()

,Unnamed: 0,project_name,project_ID,cell_index,source,output_type,execution_count,LOC,BLC,UDF,...,NBD,OPRND,OPRATOR,UOPRND,UOPRATOR,ID,ALID,MLID,LOCom,CW
0,0,kerneler_starter-portrait-segmentation-b08b391...,11413234,3.0,from mpl_toolkits.mplot3d import Axes3D\nfrom ...,NaN,1.0,7,0,0,...,4.714286,4,1,4,1,30,5,14,4,13
1,1,kerneler_starter-portrait-segmentation-b08b391...,11413234,5.0,"for dirname, _, filenames in os.walk('/kaggle/...",stream,2.0,4,0,0,...,5.250000,8,4,8,2,15,5,9,0,0
2,2,kerneler_starter-portrait-segmentation-b08b391...,11413234,7.0,# Distribution graphs (histogram/bar graph) of...,NaN,3.0,22,0,1,...,10.363636,63,29,60,9,105,5,25,2,20
3,3,kerneler_starter-portrait-segmentation-b08b391...,11413234,8.0,# Correlation matrix\ndef plotCorrelationMatri...,NaN,4.0,18,0,1,...,7.722222,49,19,46,7,103,5,21,3,16
4,4,kerneler_starter-portrait-segmentation-b08b391...,11413234,9.0,# Scatter and density plots\ndef plotScatterMa...,NaN,5.0,17,0,1,...,9.823529,47,28,45,6,118,5,17,5,41


In [225]:
Code_df.drop(columns=['Unnamed: 0']).to_csv('D:/Th/data/KGT_metrics/Code_cell_metrics.csv')

In [229]:
del Code_df

# Markdown cell metrics

In [227]:
md_df = pd.read_csv('D:/Th/data/KGT_with_no_output/KGT_Markdown.csv')
md_df.fillna('', inplace=True)

# function for count headers (H1, H2 , H3)
def header1_counter(text):
    count = 0
    if text[0:2] =='# ':
        count += 1
    for i in range(len(text)-2):
        if text[i:i+3] ==' # ' or text[i:i+3] == '\n# ':
            count +=1
    return count
def header2_counter(text):
    count = 0
    if text[0:3] =='## ':
        count += 1
    for i in range(len(text)-2):
        if text[i:i+4] ==' ## ' or text[i:i+4] == '\n## ':
            count +=1
    return count
def header3_counter(text):
    count = 0
    if text[0:4] =='### ':
        count += 1
    for i in range(len(text)-3):
        if text[i:i+5] ==' ### ' or text[i:i+5] == '\n### ':
            count +=1
    return count

In [242]:
def count_md_word(text):
    return len(text.split())

In [2]:
md_df['LMC'] = md_df['source'].apply(lambda x: len(re.findall('\n', x))+1 if type(x)==str else 0)

In [228]:
md_df['H1'] = md_df['source'].progress_apply(header1_counter)

100%|██████████████████████████████████████████████████████████████████████| 2439853/2439853 [04:35<00:00, 8840.20it/s]


In [230]:
md_df['H2'] = md_df['source'].progress_apply(header2_counter)

100%|██████████████████████████████████████████████████████████████████████| 2439853/2439853 [04:22<00:00, 9288.21it/s]


In [231]:
md_df['H3'] = md_df['source'].progress_apply(header3_counter)

100%|██████████████████████████████████████████████████████████████████████| 2439853/2439853 [04:20<00:00, 9350.14it/s]


In [244]:
md_df['MW'] = md_df['source'].progress_apply(lambda x: count_md_word(str(x)))

100%|████████████████████████████████████████████████████████████████████| 2439853/2439853 [00:13<00:00, 185070.83it/s]


In [ ]:
md_df['LMC'] = md_df['source'].apply(lambda x: )

In [251]:
md_df.head()

,Unnamed: 0,project_name,project_ID,cell_index,source,H1,H2,H3,MW,LOCom
0,0,kerneler_starter-portrait-segmentation-b08b391...,11413234.0,1,## Introduction\nGreetings from the Kaggle bot...,0,1,0,46,2
1,1,kerneler_starter-portrait-segmentation-b08b391...,11413234.0,2,## Exploratory Analysis\nTo begin this explora...,0,1,0,42,2
2,2,kerneler_starter-portrait-segmentation-b08b391...,11413234.0,4,There is 1 csv file in the current version of ...,0,0,0,12,2
3,3,kerneler_starter-portrait-segmentation-b08b391...,11413234.0,6,The next hidden code cells define functions fo...,0,0,0,24,1
4,4,kerneler_starter-portrait-segmentation-b08b391...,11413234.0,10,Now you're ready to read in the data and use t...,0,0,0,17,1


In [245]:
md_df.drop(columns=['Unnamed: 0']).to_csv('D:/Th/data/KGT_metrics/markdown_cell_metrics.csv')